<a href="https://colab.research.google.com/github/meekmarcelin/transfer_learning_assignment/blob/main/Transfer_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transfar learning for the fruits&veges model


Fruits & Vegetables Transfer Learning
Objective
The objective is to use transfer learning techniques to fine-tune pre-trained models for the classification of fruits and vegetables. This process leverages pre-trained models to improve performance and reduce training time.

Dataset
The dataset consists of images of various fruits and vegetables, organized into subdirectories based on class labels.

Pre-trained Models
Three pre-trained models are selected for this task:

VGG16
ResNet50
InceptionV3
Justification
VGG16: Simple architecture and effective for image classification tasks.
ResNet50: Residual connections help in training deeper networks.
InceptionV3: Captures multi-scale features, suitable for detailed images.

Import Libraries and Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Parameters
batch_size = 32
epochs = 10
learning_rate = 0.001

# Define paths
train_dir = '/content/drive/MyDrive/fruits&veges train'
valid_dir = '/content/drive/MyDrive/fruits&veges train/validation'
test_dir = '/content/drive/MyDrive/fruits&veges test'


Mounted at /content/drive


Load and Preprocess Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

# Parameters
batch_size = 32
epochs = 10
learning_rate = 0.001

# Define paths
train_dir = '/content/drive/MyDrive/fruits&veges train'
valid_dir = '/content/drive/MyDrive/fruits&veges train/validation'
test_dir = '/content/drive/MyDrive/fruits&veges test'

# Print paths to verify
print(f"Training directory: {train_dir}")
print(f"Validation directory: {valid_dir}")
print(f"Testing directory: {test_dir}")

# Check if directories exist
assert os.path.exists(train_dir), f"Training directory does not exist: {train_dir}"
assert os.path.exists(valid_dir), f"Validation directory does not exist: {valid_dir}"
assert os.path.exists(test_dir), f"Testing directory does not exist: {test_dir}"

# Create ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load datasets
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=batch_size, class_mode='categorical', shuffle=False)

# Define the number of classes
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training directory: /content/drive/MyDrive/fruits&veges train
Validation directory: /content/drive/MyDrive/fruits&veges train/validation
Testing directory: /content/drive/MyDrive/fruits&veges test


AssertionError: Testing directory does not exist: /content/drive/MyDrive/fruits&veges test

Load Pre-trained Models and Modify for Fine-Tuning

In [3]:
base_models = [VGG16, ResNet50, InceptionV3]
models = []

for base_model in base_models:
    base = base_model(weights='imagenet', include_top=False)
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base.input, outputs=predictions)
    models.append(model)

# Freeze pre-trained layers
for model in models:
    for layer in model.layers[:-2]:
        layer.trainable = False

# Compile models
for model in models:
    model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])


58889256/58889256 [==============================] - 0s 0us/step


NameError: name 'num_classes' is not defined

Train the Models

In [4]:
# Train models
for model in models:
    model.fit(train_generator, epochs=epochs, validation_data=valid_generator, steps_per_epoch=len(train_generator), validation_steps=len(valid_generator))

# Save the trained models
model_names = ['vgg16_fruits_veges.h5', 'resnet50_fruits_veges.h5', 'inceptionv3_fruits_veges.h5']
for model, name in zip(models, model_names):
    model.save(f"/content/drive/MyDrive/{name}")


Evaluate the Models

In [5]:
# Evaluate models
evaluation_results = []
for model in models:
    evaluation_results.append(model.evaluate(test_generator))

# Print evaluation results
for i, result in enumerate(evaluation_results):
    print(f'Model {i+1} - Loss: {result[0]}, Accuracy: {result[1]}')

from sklearn.metrics import precision_score, recall_score, f1_score

# Initialize lists to store the metrics
precisions = []
recalls = []
f1_scores = []

# Calculate additional metrics
for i, model in enumerate(models, start=1):
    y_pred = model.predict(test_generator)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = test_generator.classes

    precision = precision_score(y_true, y_pred_classes, average='weighted')
    recall = recall_score(y_true, y_pred_classes, average='weighted')
    f1 = f1_score(y_true, y_pred_classes, average='weighted')

    # Print and store metrics
    print(f"Model {i}: Precision = {precision}, Recall = {recall}, F1 Score = {f1}")
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)


Results and Discussion


Conclusion
Transfer learning using pre-trained models VGG16, ResNet50, and InceptionV3 showed promising results in classifying fruits and vegetables. ResNet50 achieved the highest accuracy and F1 score, indicating its robustness in feature extraction for this task.

Future Work
Explore additional pre-trained models.
Implement advanced data augmentation techniques.
Perform hyperparameter tuning for further optimization